In [7]:
import pandas as pd

In [13]:
import os

#load ../ck_metrics_output/class.csv as a dataframe
df1 = pd.read_csv('../ck_metrics_output/class.csv')

# load ../dataset/java/elasticsearch/dataset.csv
df2 = pd.read_csv('../dataset/java/elasticsearch/dataset.csv')

# match file names in both dfs. But first,
# remove the path up until elasticsearch folder in df1['file']
# and split the df1['fileschanged'] by 'CAS_DELIMITER'. There might be preceding and trailing commas, so remove them
# then match every file in that splitted array with df2['file']
# and create a new df with the matched files

# Remove the path up until elasticsearch folder in df1['file']
df1['file'] = df1['file'].apply(lambda x: os.path.relpath(x, start='C:\\Coding\\JIT-SDP\\repositories\\elasticsearch'))
# Replace '\' with '/' in df1['file']
df1['file'] = df1['file'].apply(lambda x: x.replace('\\', '/'))

# Save df1 file column to a new file
df1[['file']].to_csv('df1_files.csv', index=False)

# Split the df1['fileschanged'] by 'CAS_DELIMITER' and remove preceding and trailing commas
df2['fileschanged'] = df2['fileschanged'].fillna('').apply(lambda x: [f.strip(',') for f in x.split('CAS_DELIMITER') if f.strip(',')])

# Save df2 fileschanged column to a new file
df2[['fileschanged']].to_csv('df2_fileschanged.csv', index=False)

# Match every file in that splitted array with df2['file']
matched_files = df2.explode('fileschanged').merge(df1, left_on='fileschanged', right_on='file')

# Create a new dataframe with the matched files
matched_df = matched_files[['file', 'class', 'type', 'cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*', 'tcc', 'lcc', 'totalMethodsQty', 'staticMethodsQty', 'publicMethodsQty', 'privateMethodsQty', 'protectedMethodsQty', 'defaultMethodsQty', 'visibleMethodsQty', 'abstractMethodsQty', 'finalMethodsQty', 'synchronizedMethodsQty', 'totalFieldsQty', 'staticFieldsQty', 'publicFieldsQty', 'privateFieldsQty', 'protectedFieldsQty', 'defaultFieldsQty', 'finalFieldsQty', 'synchronizedFieldsQty', 'nosi', 'loc', 'returnQty', 'loopQty', 'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty', 'stringLiteralsQty', 'numbersQty', 'assignmentsQty', 'mathOperationsQty', 'variablesQty', 'maxNestedBlocksQty', 'anonymousClassesQty', 'innerClassesQty', 'lambdasQty', 'uniqueWordsQty', 'modifiers', 'logStatementsQty']]

print(matched_df)

                                                     file  \
0       server/src/main/java/org/elasticsearch/cluster...   
1       server/src/main/java/org/elasticsearch/cluster...   
2       server/src/main/java/org/elasticsearch/cluster...   
3       server/src/main/java/org/elasticsearch/cluster...   
4       server/src/main/java/org/elasticsearch/cluster...   
...                                                   ...   
417737  plugins/analysis-stempel/src/main/java/org/ela...   
417738  plugins/analysis-stempel/src/test/java/org/ela...   
417739  plugins/analysis-stempel/src/test/java/org/ela...   
417740  plugins/analysis-smartcn/src/main/java/org/ela...   
417741  plugins/analysis-kuromoji/src/main/java/org/el...   

                                                    class        type  cbo  \
0       org.elasticsearch.cluster.coordination.Followe...   anonymous    3   
1       org.elasticsearch.cluster.coordination.Followe...       class   21   
2       org.elasticsearch.cluster

In [19]:
# for every file in df1['file'], get its metrics from df2 from matching files.
# if there are multiple matching files, get the average value of the metrics
# Merge those into a new dataframe and save it as a csv file
# Group by 'file' and calculate the mean of the metrics
metrics_columns = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*', 'tcc', 'lcc', 
                   'totalMethodsQty', 'staticMethodsQty', 'publicMethodsQty', 'privateMethodsQty', 'protectedMethodsQty', 
                   'defaultMethodsQty', 'visibleMethodsQty', 'abstractMethodsQty', 'finalMethodsQty', 'synchronizedMethodsQty', 
                   'totalFieldsQty', 'staticFieldsQty', 'publicFieldsQty', 'privateFieldsQty', 'protectedFieldsQty', 
                   'defaultFieldsQty', 'finalFieldsQty', 'synchronizedFieldsQty', 'nosi', 'loc', 'returnQty', 'loopQty', 
                   'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty', 'stringLiteralsQty', 'numbersQty', 'assignmentsQty', 
                   'mathOperationsQty', 'variablesQty', 'maxNestedBlocksQty', 'anonymousClassesQty', 'innerClassesQty', 
                   'lambdasQty', 'uniqueWordsQty', 'modifiers', 'logStatementsQty']

# Merge df1 and df2 on 'file'. Have all attributes from both tables
merged_df = df1.merge(df2.explode('fileschanged'), left_on='file', right_on='fileschanged', how='left')

# Save the merged df to a csv file
merged_df.to_csv('merged_df.csv', index=False)